In [ ]:
from importlib import reload
from Files import constants, functions  # Import modules

# Reload modules
reload(constants)
reload(functions)

from Files.constants import *
from Files.functions import *

In [ ]:
driver = start_browser()

In [ ]:
login(driver,email=my_email,password=password,login_page_link=login_page)

In [ ]:
import openpyxl
import os
import time
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def create_excel():
    wb = openpyxl.Workbook()
    wb.remove(wb.active)
    attorney_ws = wb.create_sheet("Attorney Profiles")
    ppc_ws = wb.create_sheet("PPC Ads")
    attorney_ws.append(['Search Term', 'Attorney Name', 'Star Rating', 'Number of Reviews'])
    ppc_ws.append(['Search Term', 'Law Firm Name', 'Website URL', 'Screenshot File'])
    return wb, attorney_ws, ppc_ws

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def accept_cookies(driver):
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept')]"))
        )
        accept_button.click()
    except:
        pass

def scrape_attorney_profiles(driver):
    try:
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'More businesses')]"))
        )
        driver.execute_script("arguments[0].click();", more_button)
        time.sleep(1)
    except:
        pass

    profiles = []
    try:
        containers = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'Nv2PK')]"))
        )
    except:
        return profiles

    for container in containers:
        try:
            name = container.find_element(By.XPATH, ".//div[contains(@class, 'qBF1Pd')]").text
            rating = container.find_element(By.XPATH, ".//span[@aria-label]").get_attribute('aria-label').split()[1]
            reviews = container.find_element(By.XPATH, ".//span[contains(@class, 'UY7F9')]").text
            profiles.append((name, rating, reviews))
        except Exception as e:
            continue
    return profiles

def scrape_ppc_ads(driver, search_term, ppc_ws, screenshot_dir):
    try:
        ad_containers = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[.//span[text()='Ad']]"))
        )
    except:
        return

    for idx, ad in enumerate(ad_containers[:3]):
        try:
            title = ad.find_element(By.XPATH, ".//h3").text
            url = ad.find_element(By.XPATH, ".//cite").text
            driver.execute_script("arguments[0].scrollIntoView(true);", ad)
            time.sleep(0.5)
            safe_term = search_term.replace(" ", "_").replace("'", "")
            screenshot_path = f"{screenshot_dir}/{safe_term}_ad_{idx+1}.png"
            ad.screenshot(screenshot_path)
            ppc_ws.append([search_term, title, url, screenshot_path])
        except:
            continue

def main():
    cities = ["New York", "Los Angeles", "Chicago"]
    profile_terms = [f"lawyer in {city}" for city in cities] + ["attorney near me"]
    ppc_terms = [f"best law firm in {city}" for city in cities] + ["personal injury lawyer near me"] + [f"divorce attorney in {city}" for city in cities]
    
    wb, attorney_ws, ppc_ws = create_excel()
    screenshot_dir = "ppc_screenshots"
    os.makedirs(screenshot_dir, exist_ok=True)

    driver = init_driver()
    
    try:
        # Process profile searches
        for term in profile_terms:
            encoded_term = urllib.parse.quote(term)
            driver.get(f"https://www.google.com/search?q={encoded_term}")
            accept_cookies(driver)
            print("Waiting ................")

            time.sleep(5)
            
            profiles = scrape_attorney_profiles(driver)
            for name, rating, reviews in profiles:
                attorney_ws.append([term, name, rating, reviews])

        # Process PPC searches
        for term in ppc_terms:
            encoded_term = urllib.parse.quote(term)
            driver.get(f"https://www.google.com/search?q={encoded_term}")
            accept_cookies(driver)
            print("Waiting ................")
            time.sleep(5)
            
            scrape_ppc_ads(driver, term, ppc_ws, screenshot_dir)

        wb.save("legal_data.xlsx")
        print("Scraping completed successfully!")
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import time
import openpyxl

# Setup Excel file to store data
def create_excel():
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append(['Attorney Name', 'Star Rating', 'Number of Reviews', 'Screenshot File'])
    return wb, ws
wb, ws = create_excel()

In [ ]:
def scrape_attorney_data(driver, city="New York"):
    driver.get(f"https://www.google.com/search?q=personal+injury+lawyer+in+los+angeles&sca_esv=8a5f3c529fc99551&sxsrf=AHTn8zoq2G0eDO38bYANjFtklWtzpKPUwA%3A1740075438323&ei=rnG3Z8O2E_Wrxc8PjveCyAg&ved=0ahUKEwjDxYiT7tKLAxX1VfEDHY67AIkQ4dUDCBE&uact=5&oq=personal+injury+lawyer+in+los+angeles&gs_lp=Egxnd3Mtd2l6LXNlcnAiJXBlcnNvbmFsIGluanVyeSBsYXd5ZXIgaW4gbG9zIGFuZ2VsZXMyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEdIhsgOUABYAHACeAGQAQCYAQCgAQCqAQC4AQPIAQCYAgKgAgyYAwCIBgGQBgiSBwEyoAcA&sclient=gws-wiz-serp")

    # Wait for page to load
    time.sleep(3)

    # Scroll to the "More businesses" link if available to load more results
    try:
        more_button = driver.find_element(By.XPATH, "//button[text()='More businesses']")
        more_button.click()
        time.sleep(2)
    except:
        pass

    # Scrape business profiles from the map pack section
    attorney_profiles = driver.find_elements(By.XPATH, "//div[@class='VwiC3b']")  # These are business names
    
    reviews = driver.find_elements(By.XPATH, "//span[@class='YhemCb']")  # Reviews count
    ratings = driver.find_elements(By.XPATH, "//span[@class='r9fR1b']")  # Ratings (stars)
    
    return attorney_profiles, reviews, ratings

# Function to capture PPC ad screenshot
def capture_ppc_ads(driver, city="New York"):
    driver.get(f"https://www.google.com/search?q=personal+injury+lawyer+in+los+angeles&sca_esv=8a5f3c529fc99551&sxsrf=AHTn8zoq2G0eDO38bYANjFtklWtzpKPUwA%3A1740075438323&ei=rnG3Z8O2E_Wrxc8PjveCyAg&ved=0ahUKEwjDxYiT7tKLAxX1VfEDHY67AIkQ4dUDCBE&uact=5&oq=personal+injury+lawyer+in+los+angeles&gs_lp=Egxnd3Mtd2l6LXNlcnAiJXBlcnNvbmFsIGluanVyeSBsYXd5ZXIgaW4gbG9zIGFuZ2VsZXMyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEdIhsgOUABYAHACeAGQAQCYAQCgAQCqAQC4AQPIAQCYAgKgAgyYAwCIBgGQBgiSBwEyoAcA&sclient=gws-wiz-serp")
    
    # Wait for the page to load
    time.sleep(3)

    # Capture screenshots of PPC ads (ads have the "Ad" label next to them)
    ad_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'ad_cclk')]")  # Find ad elements
    
    # Take a screenshot of each ad
    screenshot_files = []
    for i, ad in enumerate(ad_elements[:3]):  # Limit to the first 3 PPC ads
        ad.screenshot(f"ppc_ad_{i+1}.png")
        screenshot_files.append(f"ppc_ad_{i+1}.png")

    return screenshot_files

# Save data to Excel
def save_data_to_excel(ws, name, rating, reviews_count, screenshot_files):
    for i, screenshot in enumerate(screenshot_files):
        ws.append([name, rating, reviews_count, screenshot])


try:
        city = "New York"
        # Scrape attorney reviews data
        attorney_profiles, reviews, ratings = scrape_attorney_data(driver, city)
        
        for name, review, rating in zip(attorney_profiles, reviews, ratings):
            attorney_name = name.text.strip()
            review_count = review.text.strip()
            star_rating = rating.text.strip()

            # Take screenshot of PPC ads
            ppc_screenshots = capture_ppc_ads(driver, city)

            # Save to Excel
            save_data_to_excel(ws, attorney_name, star_rating, review_count, ppc_screenshots)

        # Save Excel file
        wb.save(f"attorney_reviews_{city}.xlsx")
        print("Data scraping and screenshot capture completed successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    

## Yakima Changing

In [1]:
import time
import importlib
import law_functions  # Import the module
from law_functions import Law

# Reload the module in case of changes
importlib.reload(law_functions)
SEARCH_URL = "https://www.google.com/search?q=personal+injury+lawyer+in+los+angeles"

# Create an instance of Law manually
driver = Law(teardown=False)  # Set teardown to False so it doesn't quit automatically

try:
    driver.land_first_page()
    time.sleep(20)
    
    # Call the function to scroll until the "Businesses" section is found
    print("wait is over")
    # found = driver.scroll_until_businesses_section()
    xpath="/html/body/div[3]/div/div[12]/div[1]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div/div[1]/div[2]/div/h3/g-more-link/a"
    found=driver.click_extracted_link(xpath)
    print("processed")

    if found:
        print("✅ 'Businesses' section found on the page.")
    else:
        print("❌ 'Businesses' section not found.")

    # Keep the script running indefinitely
    while True:
        time.sleep(1)

except Exception as e:
    print("An error occurred:", e)


wait is over
✅ Clicked extracted link: https://www.google.com/localservices/prolist?g2lbs=AAEPWCvdIfvpYWMLoNN06d9c5LRDZwtbwNYYVBjc_vRQjVSZPafRuUaMyU4ehqetBRoXn5BYSQSTM_295nI46HnG3aSUFvJXYw%3D%3D&hl=en-PK&gl=pk&ssta=1&q=personal+injury+lawyer+in+los+angeles&oq=personal+injury+lawyer+in+los+angeles&scp=ChtnY2lkOnBlcnNvbmFsX2luanVyeV9sYXd5ZXISVhISCRPaJ9xdx8KAEfQIRiVv3y_iGhIJMc1kAdMq3YARKjm9z9YofYMiFExvcyBBbmdlbGVzLCBDQSwgVVNBKhQN58QWFBVfrES5HQR1dxQlBe-SuTAAGhZwZXJzb25hbCBpbmp1cnkgbGF3eWVyIiVwZXJzb25hbCBpbmp1cnkgbGF3eWVyIGluIGxvcyBhbmdlbGVzKhZQZXJzb25hbCBJbmp1cnkgTGF3eWVyOmYKDS9nLzExcTNxNG12dHMSKmpvYl90eXBlX2lkOmNhdGFzdHJvcGhpY19pbmp1cnlfbGl0aWdhdGlvbhoeQ2F0YXN0cm9waGljIGluanVyeSBsaXRpZ2F0aW9uIgJlbjACPWgiPD8%3D&slp=MgBSAggCYACSAZ8CCg0vZy8xMWg1dHluNHI2Cg0vZy8xMWJ6d19fODU1CgsvZy8xdGQwNTlxYgoNL2cvMTFmdDBrdzc2MgoNL2cvMTFjMXZqazIxZAoNL2cvMTFienpuX2h4MQoLL2cvMXRmejBjbmgKCy9nLzF0Z3ZkeXJ6Cg0vZy8xMWdnejZ6M3g1Cg0vZy8xMWpnNXY4ajY2CgwvZy8xMTl2YzBkazUKCy9nLzF0ajFtc3M4Cg0vZy8xMXRkZ256czAwCg0vZy8xMWRkeWo

KeyboardInterrupt: 

In [2]:
import requests

API_KEY = "e50c8fdf6169c758136b53f7827851f38b9be1aa07531b385842ef7a47f8e6c0"
params = {
    "engine": "google",
    "q": "personal injury lawyer in Los Angeles",
    "api_key": API_KEY
}

response = requests.get("https://serpapi.com/search", params=params)
print(response.json())


{'search_metadata': {'id': '67b89968dc45246b8f249fdd', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/021ebc1beada9812/67b89968dc45246b8f249fdd.json', 'created_at': '2025-02-21 15:19:04 UTC', 'processed_at': '2025-02-21 15:19:04 UTC', 'google_url': 'https://www.google.com/search?q=personal+injury+lawyer+in+Los+Angeles&oq=personal+injury+lawyer+in+Los+Angeles&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/021ebc1beada9812/67b89968dc45246b8f249fdd.html', 'total_time_taken': 1.62}, 'search_parameters': {'engine': 'google', 'q': 'personal injury lawyer in Los Angeles', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'personal injury lawyer in Los Angeles', 'organic_results_state': 'Results for exact spelling'}, 'local_ads': {'title': 'Personal injury lawyers | Los Angeles', 'see_more_text': 'More Personal Injury Lawyers', 'link': 'https://www.google.com/localservices/prolist?g2lbs=AAEPWCv-xSmjA5n

# Final script 

In [3]:
import requests
import json

# SerpApi configuration
API_KEY = "e50c8fdf6169c758136b53f7827851f38b9be1aa07531b385842ef7a47f8e6c0"  # Replace with your SerpApi key
params = {
    "engine": "google",
    "q": "personal injury lawyer in Los Angeles",
    "api_key": API_KEY,
    "hl": "en",
    "gl": "us"
}

# Fetch search results
response = requests.get("https://serpapi.com/search", params=params)
data = response.json()

# Save the API response to a JSON file
with open("api_response.json", "w") as file:
    json.dump(data, file, indent=4)

print("API response saved to api_response.json")

API response saved to api_response.json


In [11]:
import json
import csv
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Load the saved API response
with open("api_response.json", "r") as file:
    data = json.load(file)

# Extract sponsored listings
sponsored_listings = data.get('local_ads', {}).get('ads', [])

# Configure Selenium to use built-in Chrome driver
chrome_options = Options()
#chrome_options.add_argument("--headless=new")  # Run in headless mode (no GUI)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--log-level=3")  # Suppress logs
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# No need for external ChromeDriver
driver = webdriver.Chrome(options=chrome_options)

# Create a directory for screenshots
if not os.path.exists("screenshots"):
    os.makedirs("screenshots")

try:
    for listing in sponsored_listings:
        # Extract details
        title = listing.get('title', 'N/A')
        rating = listing.get('rating', 'N/A')
        rating_count = listing.get('rating_count', 'N/A')
        phone = listing.get('phone', 'N/A')
        address = listing.get('address', 'N/A')
        website = listing.get('link', 'N/A')
        
        print(f"Title: {title}")
        print(f"Rating: {rating} ({rating_count} reviews)")
        print(f"Phone: {phone}")
        print(f"Address: {address}")
        print(f"Website: {website}")
        print("-" * 40)

        # Open Google search results page
        search_url = f"https://www.google.com/search?q={title.replace(' ', '+')}"
        driver.get(search_url)
        time.sleep(2)  # Allow page to load

        # Take a full-page screenshot
        screenshot_path = f"screenshots/{title.replace(' ', '_')}.png"
        driver.save_screenshot(screenshot_path)
        print(f"Screenshot saved: {screenshot_path}")

except Exception as e:
    print(f"Error: {e}")

finally:
    driver.quit()

# Save details into a CSV file
with open('sponsored_listings.csv', 'w', newline='') as csvfile:
    fieldnames = ['Title', 'Rating', 'Rating Count', 'Phone', 'Address', 'Website']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for listing in sponsored_listings:
        writer.writerow({
            'Title': listing.get('title', 'N/A'),
            'Rating': listing.get('rating', 'N/A'),
            'Rating Count': listing.get('rating_count', 'N/A'),
            'Phone': listing.get('phone', 'N/A'),
            'Address': listing.get('address', 'N/A'),
            'Website': listing.get('link', 'N/A')
        })

print("Sponsored listings details saved to sponsored_listings.csv")


Title: Omega Law Group, PC
Rating: 4.9 (525 reviews)
Phone: +13103614832
Address: N/A
Website: https://www.google.com/localservices/prolist?g2lbs=AAEPWCu2KsV7D1ZIR86DRPQemRgp78HKMYqAkWUA6H-20Gls1SnI6Tuc9-PCDACm88OzjEnlcysW&hl=en-US&gl=us&ssta=1&src=1&gsas=1&slp=OqQEQ2hNSTE4bm53WXZWaXdNVm9BTjdCeDBiMFN2T0Vpb0lCQkFCR2d3SWh2eTRnd0lROWJDSnVRa2c4cmoyaUFJd2dJeU5uZ0k0dVp5N2hBRklnTWFHandFU0tRZ0VFQUlhQ3dpU2tOSWxFUE93X2JrSklLbUV3cDBDTUlDTWpaNENPSlNDNFk0QlNJREdobzhCRWlvSUJCQURHZ3dJcGJ2aGlBSVFwT1BKdVFrZ2g1UG9td0l3Z0l5Tm5nSTR3NG4walFGSWdNYUdqd0VTS1FnRUVBUWFDd2pidXM0YkVMbm1fYmtKSVA2N2k1b0NNSUNNalo0Q09QX2RoWTBCU0lER2hvOEJFaW9JQkJBRkdnd0lwZWJPZ1FRUTNfWDU2UWtnaGRqdm1BSXdnSXlObmdJNGd1eTNqQUZJZ01hR2p3RVNLZ2dFRUFZYURBaWMzX3ZEQWhEWjZmSEJDU0RFZ0xLY0FqQ0FqSTJlQWppaWdKbU9BVWlBeG9hUEFSSXFDQVFRQnhvTUNPS0k0WUFDRVA2ajJyZ0pJSk9zdzVrQ01JQ01qWjRDT0luVzRZd0JTSURHaG84QkVpb0lCQkFJR2d3SXBhWC0yZ1FRZ1liOGlBNGdwYnUwbGdJd2dJeU5uZ0k0MHAyYWl3RklnTWFHandFaUJnZ0VFQUlZQ0NnTU1oeVhxN0VDejgyeEFyS3BzUUxGb0xFQzFzaXhBdFd5c1FMWnlMRUM%3D